In [428]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [429]:
# Request to website and download HTML contents
url = "https://www.boxofficemojo.com/title/tt1745960/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')


In [443]:
# Find all <h3> tags and process the tables based on their titles
#movie_id = 'tt1745960'
dataframes_by_region = {}
single_release_dataframes_by_region = {}
for h3 in soup.find_all('h3'):
    if h3.get_text().strip() == 'By Release':
        table = h3.find_next('table')
        # Extract table headers
        headers = [header.get_text(strip=True) for header in table.find_all('th')]
        headers.append('ID')
        # Extract table rows
        rows = []
        for row in table.find_all('tr')[1:]:  # Skip header row
            cols = [col.get_text(strip=True) for col in row.find_all('td')]
            rows.append(cols)
        # Create a DataFrame and add to the list
        release_table_data = pd.DataFrame(rows, columns=headers)
    
    elif h3.get_text().strip() == 'By Region':
        headers = ['Country', '# Releases', 'Lifetime Gross', 'Rank', 'Region','ID']
        for table in h3.find_all_next('table'):
            # Extract the first <th> element as table name
            region_name = table.find('th').get_text(strip=True)
            # Extract table rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip header row
                cols = [col.get_text(strip=True) for col in row.find_all('td')]
                cols.append(region_name)
                cols.append(movie_id)
                rows.append(cols)
                region_table = pd.DataFrame(rows, columns=headers)
            # Create a DataFrame and store it in the dictionary with the table name
            dataframes_by_region[region_name] = region_table
            combined_df = pd.concat(dataframes_by_region.values(), ignore_index=True)
    else:
        headers = ['Country', 'Release Date', 'Opening', 'Gross', 'Region', 'ID']
        region_name_re = h3.get_text().strip()
        rows = []   
        #print(region_name_re) 
        for table in h3.find_all_next('table'):       
            for row in table.find_all('tr')[1:]:
                cols = [col.get_text(strip=True) for col in row.find_all('td')]
                cols.append(region_name_re)
                cols.append(movie_id)
                rows.append(cols)
            break  #Exit after the first 'table' found after 'h3'
        single_release_region_table = pd.DataFrame(rows, columns=headers) 
        single_release_dataframes_by_region[region_name_re] = single_release_region_table

        single_release_combined_df = pd.concat(single_release_dataframes_by_region.values() ,ignore_index=True)


In [447]:
single_release_combined_df.head(10)

,Country,Release Date,Opening,Gross,Region,ID
0,Domestic,"May 27, 2022","$126,707,459","$718,732,821",Domestic,tt1745960
1,United Kingdom,"May 27, 2022","$26,624,307","$103,456,474","Europe, Middle East, and Africa",tt1745960
2,France,"May 25, 2022","$9,979,892","$59,789,893","Europe, Middle East, and Africa",tt1745960
3,Germany,"May 26, 2022","$6,017,004","$39,294,559","Europe, Middle East, and Africa",tt1745960
4,Saudi Arabia,"May 26, 2022",–,"$20,100,000","Europe, Middle East, and Africa",tt1745960
5,Netherlands,"May 26, 2022","$2,079,384","$15,078,668","Europe, Middle East, and Africa",tt1745960
6,Italy,"May 25, 2022","$2,879,141","$13,995,686","Europe, Middle East, and Africa",tt1745960
7,Denmark,"May 26, 2022",–,"$12,940,497","Europe, Middle East, and Africa",tt1745960
8,Sweden,"May 25, 2022","$2,505,712","$11,372,433","Europe, Middle East, and Africa",tt1745960
9,Switzerland,"May 25, 2022","$1,600,000","$11,352,369","Europe, Middle East, and Africa",tt1745960


In [450]:
# Create write function
def scrape_top20(line):
    url=f"https://www.boxofficemojo.com/title/{line}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <h3> tags and process the tables based on their titles
    for h3 in soup.find_all('h3'):
        if h3.get_text().strip() == 'By Release':
            table = h3.find_next('table')
            # Extract table headers
            headers = [header.get_text(strip=True) for header in table.find_all('th')]
            headers.append('ID')
            # Extract table rows
            rows = []
            for row in table.find_all('tr')[1:]:  # Skip header row
                cols = [col.get_text(strip=True) for col in row.find_all('td')]
                cols.append(line)
                rows.append(cols)
            # Create a DataFrame and add to the list
            release_table_data = pd.DataFrame(rows, columns=headers)

            release_table_data.to_csv(f'/Users/beek/GitHub_Repo/boxoffice_scaping_analysis/top20_datasource/release/release_top20_{line}.csv', index=False)
    
        elif h3.get_text().strip() == 'By Region':
            dataframes_by_region = {}
            headers = ['Country', '# Releases', 'Lifetime Gross', 'Rank', 'Region', 'ID']
            for table in h3.find_all_next('table'):
                # Extract the first <th> element as table name
                region_name = table.find('th').get_text(strip=True)
                # Extract table rows
                rows = []
                for row in table.find_all('tr')[1:]:  # Skip header row
                    cols = [col.get_text(strip=True) for col in row.find_all('td')]
                    cols.append(region_name)
                    cols.append(line)
                    rows.append(cols)
                    region_table = pd.DataFrame(rows, columns=headers)
                # Create a DataFrame and store it in the dictionary with the table name
                dataframes_by_region[region_name] = region_table
                combined_df = pd.concat(dataframes_by_region.values(), ignore_index=True)

            combined_df.to_csv(f'/Users/beek/GitHub_Repo/boxoffice_scaping_analysis/top20_datasource/region/region_top20_{line}.csv', index=False)

        else:
            headers = ['Country', 'Release Date', 'Opening', 'Gross', 'Region', 'ID']
            region_name_re = h3.get_text().strip()
            rows = []   
            for table in h3.find_all_next('table'):       
                for row in table.find_all('tr')[1:]:
                    cols = [col.get_text(strip=True) for col in row.find_all('td')]
                    cols.append(region_name_re)
                    cols.append(line)
                    rows.append(cols)
                break  #Exit after the first 'table' found after 'h3'
            single_release_region_table = pd.DataFrame(rows, columns=headers) 
            single_release_dataframes_by_region[region_name_re] = single_release_region_table
            single_release_combined_df = pd.concat(single_release_dataframes_by_region.values() ,ignore_index=True)
            single_release_combined_df.to_csv(f'/Users/beek/GitHub_Repo/boxoffice_scaping_analysis/top20_datasource/region/region_top20_{line}.csv', index=False)


In [451]:
# Open the file
with open("top20_ids.txt", "r") as file:
    # Iterate through each line
    for line in file:
        # Process each line
        print(line.strip())
        scrape_top20(line.strip())


tt0499549
tt1630029
tt1517268
tt4154796
tt0120338
tt2488496
tt4154756
tt10872600
tt0369610
tt6105098
tt0848228
tt2820852
tt1745960
tt4520988
tt2395427
tt6718170
tt1825683
tt1201607
tt2527336
tt4881806
